# Epoch test
This script 

In [49]:
# Funky shit
import os
import csv
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/libcudart.so.11.0'
from PIL import Image,ImageOps
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [50]:
# TensorFlow and tf.keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Import MNIST Fashion data

In [51]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Setup labels as vector outputs rather than digit value
train_labelsVektor = np.zeros((10, len(train_labels)))
for i in range(len(train_labels)):
    for j in range(10):
        if train_labels[i] == j:
            train_labelsVektor[j, i] += 1
train_labelsVektor = np.transpose(train_labelsVektor)

test_labelsVektor = np.zeros((10, len(test_labels)))
for i in range(len(test_labels)):
    for j in range(10):
        if test_labels[i] == j:
            test_labelsVektor[j, i] += 1
test_labelsVektor = np.transpose(test_labelsVektor)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Normalize
train_images = train_images / 255.0
test_images = test_images / 255.0

In [52]:
def dimLDA(dim, trainIMG, trainLBL, testIMG):
    trainIMG = trainIMG.reshape([np.size(trainIMG,0),np.size(trainIMG,1)*np.size(trainIMG,2)])
    testIMG = testIMG.reshape([np.size(testIMG,0),np.size(testIMG,1)*np.size(testIMG,2)])

    lda = LDA(n_components=dim)
    redTrainIMG = lda.fit_transform(trainIMG, trainLBL)
    redTestIMG = lda.transform(testIMG)
    return redTrainIMG, redTestIMG

In [53]:
def generateModel(nodes, ActivationFunctions, inputShape):
    model = tf.keras.Sequential()
    #model.add(tf.keras.layers.InputLayer(batch_size=32))
    for i in range(len(nodes)):
        model.add(tf.keras.layers.Dense(nodes[i], activation=activationFunctions))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    return model

In [54]:
def compileModel(model):
    model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
    return model

In [55]:
def trainModel(model, epochs):
    trainAccuracy = np.zeros(epochs)
    testAccuracy = np.zeros(epochs)
    for i in range(epochs):
        print('Epoch iteration: ' + str(i+1))
        with tf.device('/GPU:0'):
            history = model.fit(train_images_red, train_labelsVektor, epochs=1)
        trainAccuracy[i] = history.history["accuracy"][0]
        test_loss, test_acc = model.evaluate(test_images_red,  test_labelsVektor)
        testAccuracy[i] = test_acc
    return trainAccuracy, testAccuracy

In [56]:
def writeToCSV(nodes, dimSet, activationFunction, trainAccuracy, testAccuracy):
    with open('./LDAtest/' + 'betteNet_dim' + str(dimSet) + '.csv', 'w', encoding='UTF8') as f: 
        writer = csv.writer(f)
        writer.writerow(['Layers: ' + str(len(nodes))])
        writer.writerow(['Nodes: ' + str(nodes)])
        writer.writerow(['LDA dim: ' + str(dimSet)])
        writer.writerow(['Activation function: ' + activationFunction])
        writer.writerow(['Training accuracy', 'Test accuracy'])
        for i in range(len(trainAccuracy)):
            writer.writerow([trainAccuracy[i], testAccuracy[i]])

In [57]:
np.shape(train_images)

(60000, 28, 28)

In [58]:
# Neural network setup
nodes = [8, 16] #64, 32
activationFunctions = 'relu'
epochs = 50

# LDA dim list
dimSet = [1, 2, 3, 4, 5, 6, 7, 8, 9]

for i in range(len(dimSet)):
    # Reduce data with PCA
    train_images_red, test_images_red = dimLDA(dimSet[i], train_images, train_labels, test_images)
    #
    print("LDA dim: " + str(dimSet[i]))
    model = generateModel(nodes, activationFunctions, dimSet[i])
    model = compileModel(model)
    trainAccuracy, testAccuracy = trainModel(model, epochs)
    writeToCSV(nodes, dimSet[i], activationFunctions, trainAccuracy, testAccuracy)
#model.summary()

LDA dim: 1
Epoch iteration: 1
313/313 [==============================] - 1s 3ms/step - loss: 0.0611 - accuracy: 0.4725
Epoch iteration: 2
313/313 [==============================] - 1s 3ms/step - loss: 0.0604 - accuracy: 0.4769
Epoch iteration: 3
313/313 [==============================] - 1s 3ms/step - loss: 0.0602 - accuracy: 0.4762
Epoch iteration: 4
313/313 [==============================] - 1s 3ms/step - loss: 0.0603 - accuracy: 0.4688
Epoch iteration: 5
313/313 [==============================] - 1s 3ms/step - loss: 0.0606 - accuracy: 0.4785
Epoch iteration: 6
313/313 [==============================] - 1s 3ms/step - loss: 0.0603 - accuracy: 0.4787
Epoch iteration: 7
313/313 [==============================] - 1s 3ms/step - loss: 0.0602 - accuracy: 0.4791
Epoch iteration: 8
313/313 [==============================] - 1s 2ms/step - loss: 0.0603 - accuracy: 0.4770
Epoch iteration: 9
313/313 [==============================] - 1s 3ms/step - loss: 0.0601 - accuracy: 0.4773
Epoch iteration: 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



313/313 [==============================] - 1s 3ms/step - loss: 0.0387 - accuracy: 0.7021
Epoch iteration: 26
313/313 [==============================] - 1s 3ms/step - loss: 0.0387 - accuracy: 0.7035
Epoch iteration: 27
313/313 [==============================] - 1s 3ms/step - loss: 0.0385 - accuracy: 0.7057
Epoch iteration: 28
313/313 [==============================] - 1s 3ms/step - loss: 0.0384 - accuracy: 0.7062
Epoch iteration: 29
313/313 [==============================] - 1s 3ms/step - loss: 0.0386 - accuracy: 0.7039
Epoch iteration: 30
313/313 [==============================] - 1s 3ms/step - loss: 0.0385 - accuracy: 0.7026
Epoch iteration: 31
313/313 [==============================] - 1s 3ms/step - loss: 0.0385 - accuracy: 0.7034
Epoch iteration: 32
313/313 [==============================] - 1s 3ms/step - loss: 0.0385 - accuracy: 0.7061
Epoch iteration: 33
313/313 [==============================] - 1s 3ms/step - loss: 0.0384 - accuracy: 0.7064
Epoch iteration: 34
313/313 [==========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



313/313 [==============================] - 1s 3ms/step - loss: 0.0386 - accuracy: 0.7042
Epoch iteration: 48
313/313 [==============================] - 1s 3ms/step - loss: 0.0383 - accuracy: 0.7041
Epoch iteration: 49
313/313 [==============================] - 1s 3ms/step - loss: 0.0382 - accuracy: 0.7065
Epoch iteration: 50
313/313 [==============================] - 1s 3ms/step - loss: 0.0383 - accuracy: 0.7031
LDA dim: 5
Epoch iteration: 1
313/313 [==============================] - 1s 3ms/step - loss: 0.0355 - accuracy: 0.7379
Epoch iteration: 2
313/313 [==============================] - 1s 3ms/step - loss: 0.0349 - accuracy: 0.7385
Epoch iteration: 3
313/313 [==============================] - 1s 3ms/step - loss: 0.0343 - accuracy: 0.7443
Epoch iteration: 4
313/313 [==============================] - 1s 3ms/step - loss: 0.0341 - accuracy: 0.7470
Epoch iteration: 5
313/313 [==============================] - 1s 3ms/step - loss: 0.0340 - accuracy: 0.7486
Epoch iteration: 6
313/313 [=====

...

313/313 [==============================] - 1s 3ms/step - loss: 0.0300 - accuracy: 0.7824
Epoch iteration: 17
313/313 [==============================] - 1s 3ms/step - loss: 0.0299 - accuracy: 0.7815
Epoch iteration: 18
313/313 [==============================] - 1s 3ms/step - loss: 0.0299 - accuracy: 0.7811
Epoch iteration: 19
313/313 [==============================] - 1s 3ms/step - loss: 0.0298 - accuracy: 0.7817
Epoch iteration: 20
313/313 [==============================] - 1s 3ms/step - loss: 0.0298 - accuracy: 0.7847
Epoch iteration: 21
313/313 [==============================] - 1s 3ms/step - loss: 0.0297 - accuracy: 0.7814
Epoch iteration: 22
313/313 [==============================] - 1s 3ms/step - loss: 0.0297 - accuracy: 0.7839
Epoch iteration: 23
313/313 [==============================] - 1s 3ms/step - loss: 0.0297 - accuracy: 0.7831
Epoch iteration: 24
313/313 [==============================] - 1s 2ms/step - loss: 0.0298 - accuracy: 0.7836
Epoch iteration: 25
313/313 [==========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



313/313 [==============================] - 1s 3ms/step - loss: 0.0257 - accuracy: 0.8184
Epoch iteration: 8
313/313 [==============================] - 1s 3ms/step - loss: 0.0256 - accuracy: 0.8197
Epoch iteration: 9
313/313 [==============================] - 1s 3ms/step - loss: 0.0256 - accuracy: 0.8195
Epoch iteration: 10
313/313 [==============================] - 1s 3ms/step - loss: 0.0257 - accuracy: 0.8172
Epoch iteration: 11
313/313 [==============================] - 1s 3ms/step - loss: 0.0256 - accuracy: 0.8174
Epoch iteration: 12
313/313 [==============================] - 1s 3ms/step - loss: 0.0255 - accuracy: 0.8192
Epoch iteration: 13
313/313 [==============================] - 1s 3ms/step - loss: 0.0254 - accuracy: 0.8194
Epoch iteration: 14
313/313 [==============================] - 1s 3ms/step - loss: 0.0254 - accuracy: 0.8196
Epoch iteration: 15
313/313 [==============================] - 1s 3ms/step - loss: 0.0255 - accuracy: 0.8199
Epoch iteration: 16
313/313 [============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



313/313 [==============================] - 1s 3ms/step - loss: 0.0243 - accuracy: 0.8289


In [59]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))
!jupyter --version

json==2.0.9
numpy==1.22.3
csv==1.0
PIL.Image==9.3.0
tensorflow==2.10.0
Selected Jupyter core packages...
IPython          : 8.6.0
ipykernel        : 6.17.0
ipywidgets       : 7.7.2
jupyter_client   : 7.4.4
jupyter_core     : 4.11.2
jupyter_server   : 1.21.0
jupyterlab       : 3.5.0
nbclient         : 0.7.0
nbconvert        : 7.2.3
nbformat         : 5.7.0
notebook         : 6.5.2
qtconsole        : not installed
traitlets        : 5.5.0
